# ABS Quarterly Producer Prices Index 6427

## Python set-up

In [1]:
# system imports
import textwrap
from typing import cast

# analytic imports
import pandas as pd
import readabs as ra
from readabs import metacol

# local imports
from abs_helper import get_abs_data
from plotting import calc_growth, plot_growth_finalise

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in the notebook
SHOW = False

## Get data from ABS

In [2]:
abs_dict, meta, source, RECENT = get_abs_data("6427.0")
plot_times = None, RECENT
plot_tags = "", "-recent"

Table 64270do001_202406 has no 'Index' sheet.
Table 64270do002_202406 has no 'Index' sheet.
Table 64270do003_202406 has no 'Index' sheet.
Table 64270do004_202406 has no 'Index' sheet.


In [3]:
textwrap.wrap(", ".join(abs_dict.keys()))

['642701, 6427011, 6427012, 6427013, 6427014, 6427017, 6427018, 6427020,',
 '6427021, 6427022, 6427023, 6427024, 6427025, 6427026, 6427027,',
 '6427034, 6427035, 6427036']

## Plot

### Headline PPI for Australia

In [4]:
quarterly_phrase = "Percentage change from previous quarter"
annual_phrase = "Percentage change from corresponding quarter of previous year"
final = "Final ;  Total (Source)"

In [5]:
get = {
    (final, "642701"): "Final Demand",
    ("Coal mining", "6427011"): "Coal mining",
    ("Manufacturing division", "6427012"): "Manufacturing",
    # ('Building construction Australia', '6427017'): 'Building construction Australia',
    # ('Road freight transport ', '6427021'): ' Road freight transport ',
}

for key, label in get.items():
    select_phrase, table = key
    data = abs_dict[table]
    identifiers = []
    for phrase in annual_phrase, quarterly_phrase:
        terms = {
            table: metacol.table,
            select_phrase: metacol.did,
            phrase: metacol.did,
        }
        _, ident, units = ra.find_abs_id(meta, terms, verbose=False)
        identifiers.append(ident)

    # plot
    title = f"Growth: {label} PPI"
    for plot_from, plot_tag in zip(plot_times, plot_tags):
        p_plot_from = (
            None
            if plot_from is None
            else pd.Period(plot_from, freq=cast(pd.PeriodIndex, data.index).freq)
        )
        plot_growth_finalise(
            data[identifiers[0]],
            data[identifiers[1]],
            title=title,
            from_=p_plot_from,
            tag=plot_tag,
            rfooter=f"{source} {table}",
            lfooter="Australia: Producer Price Index. Original series",
            annotate=9,
            show=SHOW,
        )

### Lesser PPIs - no ABS pre-calculated percentages

In [6]:
get = {
    (
        "30 Building construction Australia ;",
        "6427017",
    ): "Building construction Australia",
    ("Road freight transport ", "6427021"): "Road freight transport",
    ("Employment services", "6427025"): "Employment services",
}

for key, label in get.items():
    select_phrase, table = key
    data = abs_dict[table]
    terms = {
        table: metacol.table,
        select_phrase: metacol.did,
        "Index Number": metacol.did,
    }
    _, ident, units = ra.find_abs_id(meta, terms, verbose=False)
    growth = calc_growth(abs_dict[table][ident])

    # plot
    title = f"Growth: {label} PPI"
    for plot_from, plot_tag in zip(plot_times, plot_tags):
        p_plot_from = (
            None
            if plot_from is None
            else pd.Period(plot_from, freq=cast(pd.PeriodIndex, data.index).freq)
        )
        plot_growth_finalise(
            *growth,
            title=title,
            from_=p_plot_from,
            tag=plot_tag,
            rfooter=f"{source} {table}",
            lfooter=f"Australia: Producer Price Index. Original series",
            annotate=9,
            show=SHOW,
        )

## Finished

In [7]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Tue Aug 06 2024 20:18:00

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

pandas : 2.2.2
readabs: 0.0.17

Watermark: 2.4.3



In [8]:
print("Finished")

Finished
